In [2]:
steamDF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam.csv")
steamDF.createOrReplaceTempView("steam")
steamDF.printSchema()
steamDF.show(1)

root
 |-- recommendationid: string (nullable = true)
 |-- appid: integer (nullable = true)
 |-- game: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: long (nullable = true)
 |-- author_playtime_forever: double (nullable = true)
 |-- author_playtime_last_two_weeks: integer (nullable = true)
 |-- author_playtime_at_review: integer (nullable = true)
 |-- author_last_played: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- voted_up: string (nullable = true)
 |-- votes_up: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- wri

In [6]:
steamCountDF = spark.sql("""SELECT count(*)
                            FROM steam
                             """)
steamCountDF.printSchema()
steamCountDF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|  498103|
+--------+



In [7]:
steamCountIncorrectVotesDF = spark.sql("""SELECT count(*)
                            FROM steam
                            WHERE voted_up NOT IN (0,1)
                             """)
steamCountIncorrectVotesDF.printSchema()
steamCountIncorrectVotesDF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|   56789|
+--------+



In [8]:
steamNullGameDF = spark.sql("""SELECT count(*)
                            FROM steam
                            WHERE game IS NULL
                             """)
steamNullGameDF.printSchema()
steamNullGameDF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|      40|
+--------+



In [9]:
steamSessionDF = spark.sql("""SELECT *
                            FROM steam
                            WHERE voted_up IN (0,1) 
                            AND game IS NOT NULL
                             """)
steamSessionDF.printSchema()
steamSessionDF.show()

steamSessionDF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/correct_steam")

root
 |-- recommendationid: string (nullable = true)
 |-- appid: integer (nullable = true)
 |-- game: string (nullable = true)
 |-- author_steamid: string (nullable = true)
 |-- author_num_games_owned: string (nullable = true)
 |-- author_num_reviews: long (nullable = true)
 |-- author_playtime_forever: double (nullable = true)
 |-- author_playtime_last_two_weeks: integer (nullable = true)
 |-- author_playtime_at_review: integer (nullable = true)
 |-- author_last_played: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: string (nullable = true)
 |-- timestamp_updated: string (nullable = true)
 |-- voted_up: string (nullable = true)
 |-- votes_up: string (nullable = true)
 |-- votes_funny: string (nullable = true)
 |-- weighted_vote_score: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- steam_purchase: string (nullable = true)
 |-- received_for_free: string (nullable = true)
 |-- wri

In [10]:
steamDF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/correct_steam/steam.csv")
steamDF.createOrReplaceTempView("steam")
steamDF.printSchema()
steamDF.show(1)

root
 |-- recommendationid: integer (nullable = true)
 |-- appid: integer (nullable = true)
 |-- game: string (nullable = true)
 |-- author_steamid: long (nullable = true)
 |-- author_num_games_owned: integer (nullable = true)
 |-- author_num_reviews: integer (nullable = true)
 |-- author_playtime_forever: double (nullable = true)
 |-- author_playtime_last_two_weeks: integer (nullable = true)
 |-- author_playtime_at_review: integer (nullable = true)
 |-- author_last_played: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: integer (nullable = true)
 |-- timestamp_updated: integer (nullable = true)
 |-- voted_up: integer (nullable = true)
 |-- votes_up: integer (nullable = true)
 |-- votes_funny: long (nullable = true)
 |-- weighted_vote_score: double (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: integer (nullable = true)
 |-- received_for_free: integer (nullable = true)


In [11]:
steamCountDF = spark.sql("""SELECT count(*)
                            FROM steam
                             """)
steamCountDF.printSchema()
steamCountDF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|  441058|
+--------+



In [13]:
steamCountGamesDF = spark.sql("""SELECT COUNT(DISTINCT game)
                                 FROM steam
                             """)
steamCountGamesDF.printSchema()
steamCountGamesDF.show()

root
 |-- count(DISTINCT game): long (nullable = false)



+--------------------+
|count(DISTINCT game)|
+--------------------+
|               24935|
+--------------------+



In [17]:
steamCountGamesWithoutEADF = spark.sql("""SELECT COUNT(DISTINCT game) 
                                 FROM steam
                                 WHERE appid NOT IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                             """)
steamCountGamesWithoutEADF.printSchema()
steamCountGamesWithoutEADF.show()

root
 |-- count(DISTINCT game): long (nullable = false)



+--------------------+
|count(DISTINCT game)|
+--------------------+
|               21617|
+--------------------+



In [18]:
steamCountGamesWithEADF = spark.sql("""SELECT COUNT(DISTINCT game) 
                                 FROM steam
                                 WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                             """)
steamCountGamesWithEADF.printSchema()
steamCountGamesWithEADF.show()

root
 |-- count(DISTINCT game): long (nullable = false)



+--------------------+
|count(DISTINCT game)|
+--------------------+
|                3324|
+--------------------+



In [20]:
steamAVGRatesWithoutEADF = spark.sql("""SELECT AVG(voted_up) 
                                 FROM steam
                                 WHERE appid NOT IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                             """)
steamAVGRatesWithoutEADF.printSchema()
steamAVGRatesWithoutEADF.show()

root
 |-- avg(voted_up): double (nullable = true)



+-----------------+
|    avg(voted_up)|
+-----------------+
|0.854798520194884|
+-----------------+



In [22]:
steamAVGRatesWithEADF = spark.sql("""SELECT AVG(voted_up) 
                                 FROM steam
                                 WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                             """)
steamAVGRatesWithEADF.printSchema()
steamAVGRatesWithEADF.show()

root
 |-- avg(voted_up): double (nullable = true)



+------------------+
|     avg(voted_up)|
+------------------+
|0.8707621182404035|
+------------------+



In [23]:
steamAVGRatesWithEABeforeDF = spark.sql("""SELECT AVG(voted_up) 
                                 FROM steam
                                 WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                                 AND written_during_early_access = 1   
                             """)
steamAVGRatesWithEABeforeDF.printSchema()
steamAVGRatesWithEABeforeDF.show()

root
 |-- avg(voted_up): double (nullable = true)



+------------------+
|     avg(voted_up)|
+------------------+
|0.8619422370167978|
+------------------+



In [24]:
steamAVGRatesWithEAAfterDF = spark.sql("""SELECT AVG(voted_up) 
                                 FROM steam
                                 WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                                 AND written_during_early_access = 0   
                             """)
steamAVGRatesWithEAAfterDF.printSchema()
steamAVGRatesWithEAAfterDF.show()

root
 |-- avg(voted_up): double (nullable = true)



+------------------+
|     avg(voted_up)|
+------------------+
|0.8803194262004546|
+------------------+



In [25]:
steamSessionGroupGamesWithoutEADF = spark.sql("""SELECT game, COUNT(voted_up) AS count_reviews , AVG(voted_up) AS avg_rating
                            FROM steam
                            WHERE appid NOT IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                            GROUP BY game
                            ORDER BY COUNT(game) DESC
                             """)
steamSessionGroupGamesWithoutEADF.printSchema()
steamSessionGroupGamesWithoutEADF.show()

steamSessionGroupGamesWithoutEADF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/steam_without_early_access")

root
 |-- game: string (nullable = true)
 |-- count_reviews: long (nullable = false)
 |-- avg_rating: double (nullable = true)



+--------------------+-------------+------------------+
|                game|count_reviews|        avg_rating|
+--------------------+-------------+------------------+
|    Counter-Strike 2|         2754|0.8591140159767611|
|      Stardew Valley|         2078|0.9980750721847931|
|            Terraria|         1800|0.9938888888888889|
|Tom Clancy's Rain...|         1755|0.9002849002849003|
|  Grand Theft Auto V|         1754|0.6995438996579247|
|The Witcher 3: Wi...|         1664|0.9927884615384616|
|Red Dead Redempti...|         1414|0.9356435643564357|
|         Garry's Mod|         1382|0.9971056439942113|
|       Left 4 Dead 2|         1243|               1.0|
|Euro Truck Simula...|         1215|0.9975308641975309|
|            PAYDAY 2|         1185|0.9181434599156119|
|            Warframe|         1174|0.9011925042589438|
|      Cyberpunk 2077|         1165|0.8523605150214593|
|The Elder Scrolls...|         1149|0.9103568320278503|
|        No Man's Sky|         1114|  0.86983842

In [26]:
steamSessionGroupGamesWithEADF = spark.sql("""SELECT game, COUNT(voted_up) AS count_reviews , AVG(voted_up) AS avg_rating
                            FROM steam
                            WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                            GROUP BY game
                            ORDER BY COUNT(game) DESC
                             """)
steamSessionGroupGamesWithEADF.printSchema()
steamSessionGroupGamesWithEADF.show()

steamSessionGroupGamesWithEADF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access")

root
 |-- game: string (nullable = true)
 |-- count_reviews: long (nullable = false)
 |-- avg_rating: double (nullable = true)



+--------------------+-------------+-------------------+
|                game|count_reviews|         avg_rating|
+--------------------+-------------+-------------------+
| PUBG: BATTLEGROUNDS|         2192|0.47764598540145986|
|          The Forest|         1712|  0.991822429906542|
|    Wallpaper Engine|         1623| 0.9741219963031423|
|                Rust|         1473| 0.8615071283095723|
|                DayZ|         1209| 0.6782464846980976|
|          Subnautica|         1200| 0.9991666666666666|
|ARK: Survival Evo...|         1157| 0.8150388936905791|
|            RimWorld|         1156| 0.9991349480968859|
|Don't Starve Toge...|          985| 0.9989847715736041|
|     Project Zomboid|          980| 0.9653061224489796|
|Mount & Blade II:...|          960|           0.903125|
|        Phasmophobia|          959| 0.9989572471324296|
|       7 Days to Die|          902|  0.934589800443459|
|              Mirror|          900|                1.0|
|               Squad|         

In [27]:
steamWithoutEADF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam_without_early_access/steam_without_ea.csv")
steamWithoutEADF.createOrReplaceTempView("steam_without_ea")
steamWithoutEADF.printSchema()
steamWithoutEADF.show(1)

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+----------------+-------------+------------------+
|            game|count_reviews|        avg_rating|
+----------------+-------------+------------------+
|Counter-Strike 2|         2754|0.8591140159767611|
+----------------+-------------+------------------+
only showing top 1 row



In [28]:
steamWithEADF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access/steam_with_ea.csv")
steamWithEADF.createOrReplaceTempView("steam_with_ea")
steamWithEADF.printSchema()
steamWithEADF.show(1)

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+-------------------+-------------+-------------------+
|               game|count_reviews|         avg_rating|
+-------------------+-------------+-------------------+
|PUBG: BATTLEGROUNDS|         2192|0.47764598540145986|
+-------------------+-------------+-------------------+
only showing top 1 row



In [29]:
steamCountWithoutEADF = spark.sql("""SELECT count(*)
                            FROM steam_without_ea
                             """)
steamCountWithoutEADF.printSchema()
steamCountWithoutEADF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|   21617|
+--------+



In [30]:
steamCountWithEADF = spark.sql("""SELECT count(*)
                            FROM steam_with_ea
                             """)
steamCountWithEADF.printSchema()
steamCountWithEADF.show()

root
 |-- count(1): long (nullable = false)

+--------+
|count(1)|
+--------+
|    3324|
+--------+



In [31]:
steamAvgCountRatesWithoutEADF = spark.sql("""SELECT avg(count_reviews)
                            FROM steam_without_ea
                             """)
steamAvgCountRatesWithoutEADF.printSchema()
steamAvgCountRatesWithoutEADF.show()

root
 |-- avg(count_reviews): double (nullable = true)



+------------------+
|avg(count_reviews)|
+------------------+
|15.780450571309617|
+------------------+



In [32]:
steamAvgCountRatesWithEADF = spark.sql("""SELECT avg(count_reviews)
                            FROM steam_with_ea
                             """)
steamAvgCountRatesWithEADF.printSchema()
steamAvgCountRatesWithEADF.show()

root
 |-- avg(count_reviews): double (nullable = true)

+------------------+
|avg(count_reviews)|
+------------------+
|30.063778580024067|
+------------------+



In [40]:
steamSessionGroupGamesWithEAAfterDF = spark.sql("""SELECT game, COUNT(voted_up) AS count_reviews, AVG(voted_up) AS avg_rating
                            FROM steam
                            WHERE appid IN (
                                    SELECT DISTINCT appid
                                    FROM steam
                                    WHERE written_during_early_access = 1)
                            AND written_during_early_access = 0
                            GROUP BY game
                            ORDER BY COUNT(game) DESC
                             """)
steamSessionGroupGamesWithEAAfterDF.printSchema()
steamSessionGroupGamesWithEAAfterDF.show()

steamSessionGroupGamesWithEAAfterDF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access_after")

root
 |-- game: string (nullable = true)
 |-- count_reviews: long (nullable = false)
 |-- avg_rating: double (nullable = true)



+--------------------+-------------+-------------------+
|                game|count_reviews|         avg_rating|
+--------------------+-------------+-------------------+
| PUBG: BATTLEGROUNDS|         1946|0.45477903391572455|
|    Wallpaper Engine|         1422| 0.9711673699015471|
|          The Forest|         1413| 0.9922151450813871|
|                Rust|         1171| 0.8684884713919727|
|          Subnautica|         1004|  0.999003984063745|
|            RimWorld|          942| 0.9989384288747346|
|ARK: Survival Evo...|          937|  0.855923159018143|
|Don't Starve Toge...|          924| 0.9989177489177489|
|              Mirror|          816|                1.0|
|                DayZ|          808| 0.7673267326732673|
|              Kenshi|          733| 0.9959072305593452|
|Divinity: Origina...|          710| 0.9985915492957746|
|       The Long Dark|          613| 0.9755301794453507|
|               Squad|          607|  0.914332784184514|
|            Unturned|         

In [41]:
steamWithEAAfterDF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access_after/steam_with_ea_after.csv")
steamWithEAAfterDF.createOrReplaceTempView("steam_with_ea_after")
steamWithEAAfterDF.printSchema()
steamWithEAAfterDF.show(1)

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+-------------------+-------------+-------------------+
|               game|count_reviews|         avg_rating|
+-------------------+-------------+-------------------+
|PUBG: BATTLEGROUNDS|         1946|0.45477903391572455|
+-------------------+-------------+-------------------+
only showing top 1 row



In [42]:
steamCountWithEAAfterDF = spark.sql("""SELECT count(*)
                            FROM steam_with_ea_after
                             """)
steamCountWithEAAfterDF.printSchema()
steamCountWithEAAfterDF.show()

root
 |-- count(1): long (nullable = false)

+--------+
|count(1)|
+--------+
|    1181|
+--------+



In [43]:
steamCountWithEAAfterMore30DF = spark.sql("""SELECT count(*)
                            FROM steam_with_ea_after
                            WHERE count_reviews > 30
                             """)
steamCountWithEAAfterMore30DF.printSchema()
steamCountWithEAAfterMore30DF.show()

root
 |-- count(1): long (nullable = false)

+--------+
|count(1)|
+--------+
|     232|
+--------+



In [44]:
steamCountWithoutEAMore30DF = spark.sql("""SELECT count(*)
                            FROM steam_without_ea
                            WHERE count_reviews > 30
                             """)
steamCountWithoutEAMore30DF.printSchema()
steamCountWithoutEAMore30DF.show()

root
 |-- count(1): long (nullable = false)



+--------+
|count(1)|
+--------+
|    1917|
+--------+



In [45]:
steamAvgRatingWithEAAfterMore30DF = spark.sql("""SELECT avg(avg_rating)
                            FROM steam_with_ea_after
                            WHERE count_reviews > 30
                             """)
steamAvgRatingWithEAAfterMore30DF.printSchema()
steamAvgRatingWithEAAfterMore30DF.show()

root
 |-- avg(avg_rating): double (nullable = true)



+------------------+
|   avg(avg_rating)|
+------------------+
|0.8701199984976935|
+------------------+



In [46]:
steamAvgRatingWithoutEAMore30DF = spark.sql("""SELECT avg(avg_rating)
                            FROM steam_without_ea
                            WHERE count_reviews > 30
                             """)
steamAvgRatingWithoutEAMore30DF.printSchema()
steamAvgRatingWithoutEAMore30DF.show()

root
 |-- avg(avg_rating): double (nullable = true)



+------------------+
|   avg(avg_rating)|
+------------------+
|0.8541110053684383|
+------------------+



In [52]:
steamSessionGroupGamesWithEAAfterTop100DF = spark.sql("""SELECT *
                            FROM steam_with_ea_after
                            LIMIT 100
                             """)
steamSessionGroupGamesWithEAAfterTop100DF.printSchema()
steamSessionGroupGamesWithEAAfterTop100DF.show()

steamSessionGroupGamesWithEAAfterTop100DF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access_after_top100")

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+--------------------+-------------+-------------------+
|                game|count_reviews|         avg_rating|
+--------------------+-------------+-------------------+
| PUBG: BATTLEGROUNDS|         1946|0.45477903391572455|
|    Wallpaper Engine|         1422| 0.9711673699015471|
|          The Forest|         1413| 0.9922151450813871|
|                Rust|         1171| 0.8684884713919727|
|          Subnautica|         1004|  0.999003984063745|
|            RimWorld|          942| 0.9989384288747346|
|ARK: Survival Evo...|          937|  0.855923159018143|
|Don't Starve Toge...|          924| 0.9989177489177489|
|              Mirror|          816|                1.0|
|                DayZ|          808| 0.7673267326732673|
|              Kenshi|          733| 0.9959072305593452|
|Divinity: Origina...|          710| 0.9985915492957746|
|       The Lon

In [53]:
steamSessionGroupGamesWithoutEATop100DF = spark.sql("""SELECT *
                            FROM steam_without_ea
                            LIMIT 100
                             """)
steamSessionGroupGamesWithoutEATop100DF.printSchema()
steamSessionGroupGamesWithoutEATop100DF.show()

steamSessionGroupGamesWithoutEATop100DF.coalesce(1).write.options(header = "True", delimiter=",").mode("overwrite") \
    .csv("file:/home/student/Data/steam_project/steam_without_early_access_top100")

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+--------------------+-------------+------------------+
|                game|count_reviews|        avg_rating|
+--------------------+-------------+------------------+
|    Counter-Strike 2|         2754|0.8591140159767611|
|      Stardew Valley|         2078|0.9980750721847931|
|            Terraria|         1800|0.9938888888888889|
|Tom Clancy's Rain...|         1755|0.9002849002849003|
|  Grand Theft Auto V|         1754|0.6995438996579247|
|The Witcher 3: Wi...|         1664|0.9927884615384616|
|Red Dead Redempti...|         1414|0.9356435643564357|
|         Garry's Mod|         1382|0.9971056439942113|
|       Left 4 Dead 2|         1243|               1.0|
|Euro Truck Simula...|         1215|0.9975308641975309|
|            PAYDAY 2|         1185|0.9181434599156119|
|            Warframe|         1174|0.9011925042589438|
|      Cyberpunk 2077|        

In [54]:
steamWithEAAfterTop100DF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam_with_early_access_after_top100/steam_with_ea_after_top100.csv")
steamWithEAAfterTop100DF.createOrReplaceTempView("steam_with_ea_after100")
steamWithEAAfterTop100DF.printSchema()
steamWithEAAfterTop100DF.show(1)

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+-------------------+-------------+-------------------+
|               game|count_reviews|         avg_rating|
+-------------------+-------------+-------------------+
|PUBG: BATTLEGROUNDS|         1946|0.45477903391572455|
+-------------------+-------------+-------------------+
only showing top 1 row



In [55]:
steamWithoutEATop100DF = spark.read.option("sep",",").option("header", "true").option("inferSchema",  "true") \
    .csv("file:/home/student/Data/steam_project/steam_without_early_access_top100/steam_without_ea_top100.csv")
steamWithoutEATop100DF.createOrReplaceTempView("steam_without_ea100")
steamWithoutEATop100DF.printSchema()
steamWithoutEATop100DF.show(1)

root
 |-- game: string (nullable = true)
 |-- count_reviews: integer (nullable = true)
 |-- avg_rating: double (nullable = true)

+----------------+-------------+------------------+
|            game|count_reviews|        avg_rating|
+----------------+-------------+------------------+
|Counter-Strike 2|         2754|0.8591140159767611|
+----------------+-------------+------------------+
only showing top 1 row



In [57]:
steamSumRatesWithEATop100DF = spark.sql("""SELECT sum(count_reviews)
                            FROM steam_with_ea_after100
                             """)
steamSumRatesWithEATop100DF.printSchema()
steamSumRatesWithEATop100DF.show()

root
 |-- sum(count_reviews): long (nullable = true)

+------------------+
|sum(count_reviews)|
+------------------+
|             34735|
+------------------+



In [59]:
steamSumRatesWithoutEATop100DF = spark.sql("""SELECT sum(count_reviews)
                            FROM steam_without_ea100
                             """)
steamSumRatesWithoutEATop100DF.printSchema()
steamSumRatesWithoutEATop100DF.show()

root
 |-- sum(count_reviews): long (nullable = true)

+------------------+
|sum(count_reviews)|
+------------------+
|             77986|
+------------------+



In [60]:
steamAvgRatingWithEAAfterTop100DF = spark.sql("""SELECT avg(avg_rating)
                            FROM steam_with_ea_after100
                             """)
steamAvgRatingWithEAAfterTop100DF.printSchema()
steamAvgRatingWithEAAfterTop100DF.show()

root
 |-- avg(avg_rating): double (nullable = true)

+------------------+
|   avg(avg_rating)|
+------------------+
|0.9093949622389083|
+------------------+



In [61]:
steamAvgRatingWithoutEATop100DF = spark.sql("""SELECT avg(avg_rating)
                            FROM steam_without_ea100
                             """)
steamAvgRatingWithoutEATop100DF.printSchema()
steamAvgRatingWithoutEATop100DF.show()

root
 |-- avg(avg_rating): double (nullable = true)

+----------------+
| avg(avg_rating)|
+----------------+
|0.89668220162413|
+----------------+

